In [ ]:
import sys, os
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tnrange, tqdm_notebook, tqdm
from IPython.display import clear_output

### Load the model and define a list of classes.


In [ ]:
emotions = ['disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'angry']

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0,)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 300)
        self.fc2 = nn.Linear(300, 60)
        self.fc3 = nn.Linear(60, 7)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

network=Network()
network.load_state_dict(torch.load('./model/model_f2.pt', map_location=torch.device('cpu')))

### Hard-code warning!
The display_emoji function creates a CV2 window to display an image of the emoji corresponding to the predicted class.


In [ ]:
def display_emoji(max_index):
    if max_index == 0:
        path = './emojis/disguist.png'
        im = cv2.imread(path)
        cv2.imshow('', im)

    if max_index == 1:
        path = './emojis/fear.png'
        im = cv2.imread(path)
        cv2.imshow('', im)
        
    if max_index == 2:
        path = './emojis/happy.png'
        im = cv2.imread(path)
        cv2.imshow('', im)

    if max_index == 3:
        path = './emojis/sad.png'
        im = cv2.imread(path)
        cv2.imshow('', im)

    if max_index == 4:
        path = './emojis/surprise.png'
        im = cv2.imread(path)
        cv2.imshow('', im)
        
    if max_index == 5:
        path = './emojis/neutral.png'
        im = cv2.imread(path)
        cv2.imshow('', im)

    if max_index == 6:
        path = './emojis/angry.png'
        im = cv2.imread(path)
        cv2.imshow('', im)


### End Game:
The cell below streams webcam frames through the trained model, which outputs a tensor. Then display_emoji displays the corresponding image. 

### Note :
Press 'Q' to terminate.

In [ ]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)


while True:
    
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        
    for (x,y,w,h) in faces_detected:
        
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = (roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        p_image=torch.Tensor(img_pixels)
        img_pixels //= 255
        img_pixels=torch.Tensor(img_pixels)

        with torch.no_grad():
            
            clear_output(wait=True)
            network.eval()
            #p_image=p_image.cuda()
            lab=network(p_image.unsqueeze(0))
            #lab=lab.cpu()
            max_index = np.argmax(lab[0])
            predicted_emotion = emotions[max_index] 
            display_emoji(max_index)
            cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            #plt.plot(lab[0])

    resized_img = cv2.resize(test_img, (700, 500))
    cv2.imshow('Facial-expression-clasifier',resized_img)

    if cv2.waitKey(10) == ord('q'):
        cv2.destroyAllWindows()
        break